# Titanic death predicion using decision trees

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

test.csv
train.csv



In [ ]:
# Baseline models

baseline = {"tautis": 0.76, "always no": 0.616}

In [29]:
# load data from kaggle
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [53]:
# local testing 

full_dataset = pd.read_csv("../input/titanic.csv")

## make format similar to kaggle
full_dataset.drop(['boat','body', 'home.dest'], axis=1, inplace=True)
full_dataset.rename(columns={
    'pclass': 'Pclass', 'survived': 'Survived', "name": "Name", "sex":"Sex",
    "age":"Age", "sibsp":"SibSp", "parch":"Parch", "ticket": "Ticket",
    "fare":"Fare", "cabin":"Cabin", "embarked":"Embarked"
}, inplace=True)

from sklearn.model_selection import train_test_split

train, test, _, _ = train_test_split(full_dataset, full_dataset, test_size=0.32, random_state=0)

def localTest(prediction):
    "localy test prediction accuracy on test score"
    assert(len(prediction) == len(test))
    return (test.Survived == prediction).sum() / len(test)

# Plan 
 1. Understand data
 2. Split the data - I think this is already done for me 
 3. Feature engineer 
 4. Data generator
 5. Run few decision tree models. I want to try decision trees, random trees, boosted methods. All must run on the same data. 
 

In [ ]:
train.head()

In [ ]:
train.Embarked.unique()

## Observations:
 1. Break up into separate vectors `Pclass` ,`Embarked`
 2. Drop `Name` , `PassengerId`, `Survived`, `Ticket`
 3. Split `Cabin` into a letter vectors and a number
 4. Binarize `Sex`

In [3]:
def breakupIntoVectors(row, prefix):
    """
    creates separate classes for predictor
    if nan, propagate nan
    """
    unique = row.unique()
    mask = unique == unique
    unique = unique[mask]
    new_df = {}
    for key in unique:
        new_df[prefix + str(key)] = row == key
        new_df[prefix + str(key)][row.isnull()] = np.nan
    
    return pd.DataFrame(new_df)
        

def binarizeSex(df):
    "male = 1, female = 0"
    row = df.Sex
    res = row == "male"
    res[row.isnull()] = np.nan
    return pd.DataFrame( {"Sex": res})

def splitCabin(df):
    "break up cabin into letter and number"
    row = df.Cabin
    # to do

In [4]:
def prepareDataset(df):
    "prepare structure"
    df_pclass = breakupIntoVectors(df.Pclass, "Pclass_")
    df_Embarked = breakupIntoVectors(df.Embarked, "Embarked_")
    df_sex = binarizeSex(df)
    df_others = pd.DataFrame({
        "Age" : df.Age,
        "SibSp" : df.SibSp,
        "Parch" : df.Parch,
        "Fare" : df.Fare,
        "FareLog" : np.log(df.Fare+1)
    })
    return pd.concat([df_pclass, df_Embarked, df_sex, df_others], axis=1)

import random

def randomiseMissingData(df2):
    "randomise missing data on pandas"
    df = df2.copy()
    for col in df.columns:
        data = df[col]
        mask = data.isnull()
        samples = random.choices( data[~mask].values , k = mask.sum() )
        data[mask] = samples
    
    return df


def prepareLabels(df):
    "prepare labels"
    return df.Survived

In [8]:
def saveSubmisionFile(prediction, filename):
    "saves submission output, assuming that predictions come in the same sequence and test df"
    assert(len(prediction) == len(test))
    out = pd.DataFrame({
        "PassengerId" : test.PassengerId.values,
        "Survived" : prediction.astype(int)
    })
    print("save file: "+ filename+ " with shape:",out.shape)
    out.to_csv( filename , index = False )
    
def runPredictionsFor(n):
    "run predictions for n times"
    res = [ predict() for i in range(n)]
    return np.apply_along_axis( func1d=np.median, arr=np.vstack(res), axis=0)

Due to random filling of missing data, we must run our algorithms multiple times to get an unbiased prediction.  

In [59]:
# simple decision tree

def predict(): 
    from sklearn import tree
    global clf
    
    X = randomiseMissingData(prepareDataset(train))
    Y = prepareLabels(train)

    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X, Y)

    X_test = randomiseMissingData(prepareDataset(test))
    return clf.predict(X_test)

out1 = runPredictionsFor(1)
print("simple_tree_runs_1", localTest(out1)) # 0.7446

#saveSubmisionFile(out1, "simple_tree_runs_1.csv")
# results on kaggle test set: 0.70813

out100 = runPredictionsFor(100)
print("simple_tree_runs_100", localTest(out100)) # 0.7708

#saveSubmisionFile(out100, "simple_tree_runs_100.csv")
# results on kaggle test set: 0.72727

/home/kmisiunas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


simple_tree_runs_1 0.744630071599
simple_tree_runs_100 0.770883054893


In [60]:
list(zip(X.keys(),clf.feature_importances_) )

[('Pclass_1', 0.014338316687684654),
 ('Pclass_2', 0.003733659993623413),
 ('Pclass_3', 0.080565638777347265),
 ('Embarked_C', 0.0095446947205410602),
 ('Embarked_Q', 0.0066884285073721978),
 ('Embarked_S', 0.01560213383185452),
 ('Sex', 0.29547726937755259),
 ('Age', 0.2389043399029615),
 ('Fare', 0.10058526805468999),
 ('FareLog', 0.15702541894907676),
 ('Parch', 0.03718173997722965),
 ('SibSp', 0.04035309122006648)]

In [ ]:
# Random forrests 
def predict(): 
    from sklearn.ensemble import RandomForestClassifier
    global clf, X, Y
    
    X = randomiseMissingData(prepareDataset(train))
    Y = prepareLabels(train)

    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf = clf.fit(X, Y)

    X_test = randomiseMissingData(prepareDataset(test))
    return clf.predict(X_test)


out1 = runPredictionsFor(1)
print("random_forests_1_runs_1", localTest(out1)) # 0.735

#saveSubmisionFile(out1, "random_forests_1_runs_1.csv")
# results on kaggle test set: 0.72248

out100 = runPredictionsFor(100)
print("random_forests_1_runs_100", localTest(out100)) # 0.735

#saveSubmisionFile(out100, "random_forests_1_runs_100.csv")
# results on kaggle test set: 0.72248

a surprising result is that `out1 == out100` for random forests. This was not the case for the decision tree. There might be a bug somewhere

In [19]:
list(zip(X.keys(),clf.feature_importances_) )

[('Pclass_1', 0.15325717017649079),
 ('Pclass_2', 0.0),
 ('Pclass_3', 0.12796425192666278),
 ('Embarked_C', 0.017696833313819126),
 ('Embarked_Q', 0.0017106614995323506),
 ('Embarked_S', 0.010501231225105389),
 ('Sex', 0.33586081249295024),
 ('Age', 0.011438564695887051),
 ('Fare', 0.095696470835129402),
 ('FareLog', 0.19202316241156819),
 ('Parch', 0.053850841422854664),
 ('SibSp', 0.0)]

In [ ]:
# xgBoost
def predict(): 
    from xgboost import XGBClassifier
    global clf, X, Y

    X = randomiseMissingData(prepareDataset(train))
    Y = prepareLabels(train) 

    clf = XGBClassifier()
    clf.fit(X, Y)
    
    X_test = randomiseMissingData(prepareDataset(test))
    return clf.predict(X_test)

out1 = runPredictionsFor(1)
print("xgboost_1_runs_1", localTest(out1)) # 0.8019093

#saveSubmisionFile(out1, "xgboost_1_runs_1.csv")
# results on kaggle test set: 0.77511

out100 = runPredictionsFor(100)
print("xgboost_1_runs_100", localTest(out100)) # 0.80190

#saveSubmisionFile(out100, "xgboost_1_runs_100.csv")
# results on kaggle test set: 

In [57]:
print(clf)
list(zip(X.keys(),clf.feature_importances_) )

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)


[('Pclass_1', 0.17219018794097454),
 ('Pclass_2', 0.0),
 ('Pclass_3', 0.14872151114990478),
 ('Embarked_C', 0.0026618471101808815),
 ('Embarked_Q', 0.0),
 ('Embarked_S', 0.0),
 ('Sex', 0.35740188103811665),
 ('Age', 0.010275356821308495),
 ('Fare', 0.090821787175992791),
 ('FareLog', 0.15914369578151835),
 ('Parch', 0.058783732982003456),
 ('SibSp', 0.0)]

## To do

If accuracy was the aim, we could make better use of the data:
 - use name as a proxy for class
 - use cabin position as a atribute (see above)